<a href="https://colab.research.google.com/github/Tuhinansh/Bone-Fraction-detection-project-/blob/main/Bone_Detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import zipfile
import os

zip_path = "/content/Dataset_Bone.zip"
extract_path = "/content/Bone"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully")

# Show folder structure
for root, dirs, files in os.walk(extract_path):
    level = root.replace(extract_path, "").count(os.sep)
    indent = " " * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * 4 * (level + 1)
    for f in files[:5]:
        print(f"{subindent}{f}")


Dataset extracted successfully
Bone/
    Dataset/
        val/
            fractured/
                4-rotated3-rotated3-rotated1.jpg
                7-rotated2-rotated3-rotated1.jpg
                6-rotated1-rotated2.jpg
                8-rotated2-rotated1.jpg
                3-rotated3-rotated3-rotated1.jpg
            not-fractured/
                2-rotated3-rotated1-rotated2-rotated1.jpg
                3-rotated3-rotated3-rotated1.jpg
                2-rotated2-rotated2.jpg
                4-rotated2-rotated2-rotated3.jpg
                3-rotated1-rotated1-rotated1.jpg
        train/
            fractured/
                88-rotated2-rotated3-rotated2.jpg
                59-rotated1-rotated3.jpg
                102-rotated3-rotated3-rotated3.jpg
                105-rotated1-rotated2-rotated1.jpg
                77-rotated3-rotated2.jpg
            not-fractured/
                7-rotated3-rotated3-rotated1-rotated1.jpg
                25-rotated3-rotated2-rotated1-rotated1.jpg

In [4]:
train_dir = "/content/Bone/Dataset/train"
val_dir = "/content/Bone/Dataset/val"

IMG_SIZE = 224
BATCH_SIZE = 32

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    zoom_range=0.05,
    brightness_range=[0.9,1.1],
    horizontal_flip=True
)

val_gen = ImageDataGenerator(
    rescale=1./255
)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_data = val_gen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

print(train_data.class_indices)


Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
{'fractured': 0, 'not-fractured': 1}


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, Model

IMG_SIZE = 224


In [6]:
base_model = tf.keras.applications.DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling='avg'
)

# Freeze base initially
base_model.trainable = False

x = layers.Dropout(0.4)(base_model.output)
output = layers.Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,408 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 56, 56,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 56, 56,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 56, 56,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 56, 56,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 56, 56,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 56, 56,    │     12,288 │ conv2_block2_0_r

 Total params: 7,038,529 (26.85 MB)

 Trainable params: 1,025 (4.00 KB)

 Non-trainable params: 7,037,504 (26.85 MB)

In [7]:
history1 = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
277/277 ━━━━━━━━━━━━━━━━━━━━ 191s 588ms/step - accuracy: 0.4941 - loss: 0.8621 - val_accuracy: 0.5333 - val_loss: 0.6995
Epoch 2/5
277/277 ━━━━━━━━━━━━━━━━━━━━ 121s 436ms/step - accuracy: 0.5122 - loss: 0.7941 - val_accuracy: 0.5983 - val_loss: 0.6431
Epoch 3/5
277/277 ━━━━━━━━━━━━━━━━━━━━ 123s 443ms/step - accuracy: 0.5639 - loss: 0.7309 - val_accuracy: 0.5850 - val_loss: 0.6161
Epoch 4/5
277/277 ━━━━━━━━━━━━━━━━━━━━ 123s 443ms/step - accuracy: 0.5906 - loss: 0.6891 - val_accuracy: 0.6350 - val_loss: 0.5823
Epoch 5/5
277/277 ━━━━━━━━━━━━━━━━━━━━ 121s 437ms/step - accuracy: 0.6142 - loss: 0.6597 - val_accuracy: 0.6200 - val_loss: 0.5854


In [ ]:
for layer in base_model.layers[-50:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history2 = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)


Epoch 1/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 195s 578ms/step - accuracy: 0.5890 - loss: 0.7453 - val_accuracy: 0.7183 - val_loss: 0.5039
Epoch 2/10


In [ ]:
val_loss, val_acc = model.evaluate(val_data)
print("Validation Accuracy:", val_acc)


In [ ]:
!pip install reportlab

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import seaborn as sns
import cv2
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image as RLImage
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
import tensorflow as tf


In [ ]:
# Get all validation images and predictions
val_data.reset()

y_true = []
y_pred = []
y_prob = []

for i in range(len(val_data)):
    x, y = val_data[i]
    preds = model.predict(x, verbose=0)

    y_true.extend(y)
    y_prob.extend(preds[:,0])
    y_pred.extend(np.where(preds[:,0] > 0.5, 1, 0))

y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_prob = np.array(y_prob)


In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['fractured','not-fractured'],
            yticklabels=['fractured','not-fractured'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig("confusion_matrix.png", dpi=200)
plt.show()


In [ ]:
fpr, tpr, _ = roc_curve(y_true, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.4f}")
plt.plot([0,1],[0,1],'--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title("ROC Curve")
plt.legend()
plt.savefig("roc_curve.png", dpi=200)
plt.show()


In [ ]:
print("\n===== CLASSIFICATION REPORT =====\n")
print(classification_report(y_true, y_pred, target_names=["fractured","not-fractured"]))

accuracy = (y_true == y_pred).mean()
print("Accuracy:", accuracy)
print("AUC:", roc_auc)


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random

# ------------------------------
# FIX: test_gen = val_data
# ------------------------------
test_gen = val_data   # your dataset has only train and val

# ------------------------------
# GRAD-CAM FUNCTION
# ------------------------------
def generate_gradcam(model, img_array, layer_name):
    grad_model = tf.keras.models.Model(
        [model.input],
        [model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_output, preds = grad_model(img_array)
        class_idx = tf.argmax(preds[0])
        loss = preds[:, class_idx]

    grads = tape.gradient(loss, conv_output)[0]
    weights = tf.reduce_mean(grads, axis=(0, 1))

    cam = np.zeros(conv_output.shape[1:3], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * conv_output[0, :, :, i]

    cam = np.maximum(cam, 0)
    cam = cam / (cam.max() + 1e-8)
    return cam


# ------------------------------
# PICK RANDOM IMAGE FROM val_data
# ------------------------------
idx = random.randint(0, len(test_gen.filepaths) - 1)
test_img_path = test_gen.filepaths[idx]
print("Using image:", test_img_path)

img = tf.keras.preprocessing.image.load_img(test_img_path, target_size=(224, 224))
img_arr = tf.keras.preprocessing.image.img_to_array(img) / 255.0
img_exp = np.expand_dims(img_arr, axis=0)

# ------------------------------
# GENERATE CAM
# ------------------------------
heatmap = generate_gradcam(model, img_exp, layer_name="conv5_block16_2_conv")

heatmap_resized = cv2.resize(heatmap, (224, 224))
heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap_resized), cv2.COLORMAP_JET)
overlay = heatmap_color * 0.4 + (img_arr * 255)

# ------------------------------
# PLOT
# ------------------------------
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.imshow(img_arr)
plt.title("Original Image")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(overlay.astype("uint8"))
plt.title("Grad-CAM Heatmap")
plt.axis("off")

plt.show()


In [ ]:
pdf_path = "BoneFracture_Evaluation_Report.pdf"
doc = SimpleDocTemplate(pdf_path, pagesize=letter)
styles = getSampleStyleSheet()
story = []

title = "<b><font size=16>Bone Fracture Detection – Model Evaluation Report</font></b>"
story.append(Paragraph(title, styles["Title"]))
story.append(Spacer(1, 16))

story.append(Paragraph("<b>1. Confusion Matrix</b>", styles["Heading2"]))
story.append(RLImage("confusion_matrix.png", width=400, height=300))
story.append(Spacer(1, 16))

story.append(Paragraph("<b>2. ROC Curve</b>", styles["Heading2"]))
story.append(RLImage("roc_curve.png", width=400, height=300))
story.append(Spacer(1, 16))

story.append(Paragraph("<b>3. Grad-CAM Examples</b>", styles["Heading2"]))
for p in gradcam_paths:
    story.append(RLImage(p, width=400, height=300))
    story.append(Spacer(1, 12))

# Add metrics summary
summary = f"""
<b>4. Metrics Summary</b><br/><br/>
Accuracy: {accuracy:.4f}<br/>
AUC-ROC: {roc_auc:.4f}<br/>
"""
story.append(Paragraph(summary, styles["Normal"]))

doc.build(story)

print("PDF Report saved as:", pdf_path)


In [ ]:
model.save("bone_fracture_model.h5")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing import image
import os
import random

# Snippet to visualize predictions with simulated fracture region bounding boxes
def visualize_predictions_with_boxes(model, data_dir, num_images=5, confidence_threshold=0.5):
    """
    Load random images from validation directory, predict with model,
    and draw bounding boxes around predicted fracture regions
    """
    all_images = []
    all_labels = []

    # Collect all image paths and labels
    for class_name in ['fractured', 'not-fractured']:
        class_path = os.path.join(data_dir, class_name)
        if os.path.exists(class_path):
            images = [os.path.join(class_path, f) for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            all_images.extend(images)
            all_labels.extend([class_name] * len(images))

    # Select random images
    if len(all_images) < num_images:
        selected_indices = list(range(len(all_images)))
    else:
        selected_indices = random.sample(range(len(all_images)), num_images)

    fig, axes = plt.subplots(1, num_images, figsize=(20, 4))
    if num_images == 1:
        axes = [axes]

    for i, idx in enumerate(selected_indices):
        img_path = all_images[idx]
        true_label = all_labels[idx]

        # Load and preprocess image
        img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0

        # Predict
        pred = model.predict(img_array)[0][0]
        pred_class = 'fractured' if pred > 0.5 else 'not-fractured'
        confidence = pred if pred > 0.5 else 1 - pred

        # Load original image for visualization (maintain aspect ratio)
        orig_img = cv2.imread(img_path)
        orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
        h, w = orig_img.shape[:2]

        # Simulate fracture bounding box (adjust these coordinates based on your needs)
        # For demo: center box covering ~30% of image area
        if pred_class == 'fractured' and confidence > confidence_threshold:
            box_h = int(h * 0.3)
            box_w = int(w * 0.3)
            x1 = int(w * 0.35)
            y1 = int(h * 0.35)
            x2 = x1 + box_w
            y2 = y1 + box_h

            # Draw bounding box
            cv2.rectangle(orig_img, (x1, y1), (x2, y2), (0, 255, 0), 3)
            cv2.putText(orig_img, f'{pred_class}: {confidence:.2f}', (x1, y1-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        else:
            cv2.putText(orig_img, f'{pred_class}: {confidence:.2f}', (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        axes[i].imshow(orig_img)
        axes[i].set_title(f'True: {true_label}\nPred: {pred_class}')
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

# Usage after model training:
# visualize_predictions_with_boxes(model, val_dir, num_images=5)


In [ ]:
import matplotlib.pyplot as plt

# Suppose you trained like:
# history = model.fit(
#     train_data,
#     validation_data=val_data,
#     epochs=20,
#     callbacks=[...]
# )

# 1) Epoch vs Loss
plt.figure(figsize=(6,4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Epoch vs Loss')
plt.legend()
plt.grid(True)
plt.show()

# 2) Epoch vs Accuracy / Precision / Recall / F1 / IoU
# Make sure you compiled the model with these metrics, e.g.:
# metrics=[tf.keras.metrics.Precision(name='precision'),
#          tf.keras.metrics.Recall(name='recall'),
#          tfa.metrics.F1Score(num_classes=1, average='micro', name='f1'),
#          tfa.metrics.FBetaScore(num_classes=1, beta=1.0, average='micro', name='iou')]

def plot_metric(history, metric_name, display_name=None):
    if display_name is None:
        display_name = metric_name.capitalize()
    plt.figure(figsize=(6,4))
    plt.plot(history.history[metric_name], label=f'Train {display_name}')
    plt.plot(history.history['val_' + metric_name], label=f'Validation {display_name}')
    plt.xlabel('Epoch')
    plt.ylabel(display_name)
    plt.title(f'Epoch vs {display_name}')
    plt.legend()
    plt.grid(True)
    plt.show()

# Call for each metric you logged during model.compile
plot_metric(history, 'precision', 'Precision')
plot_metric(history, 'recall', 'Recall')
plot_metric(history, 'f1', 'F1-score')
plot_metric(history, 'iou', 'IoU')

# 3) Train Loss vs Validation F1-score (scatter-style like bottom-left plot)
plt.figure(figsize=(6,4))
plt.scatter(history.history['loss'], history.history['val_f1'])
plt.xlabel('Train Loss')
plt.ylabel('Validation F1-score')
plt.title('Train Loss vs Validation F1-score')
plt.grid(True)
plt.show()
